# Model Context Protocol (MCP) Labs - REAL MCP Integration

## 🚀 Running on Google Colab with ACTUAL MCP Servers

**✅ USING REAL MCP SERVERS FROM mcpservers.org:**
- ✅ `mcp-google` - Gmail + Calendar MCP Server (npm package)
- ✅ `@modelcontextprotocol/server-filesystem` - Filesystem MCP Server
- ✅ **REAL MCP protocol implementation (NOT simulation!)**
- ✅ Async event loop handling with nest_asyncio
- ✅ Persistent MCP sessions throughout agent execution

### Key Changes from Previous Version:
- ❌ **REMOVED**: Direct Google API calls
- ✅ **ADDED**: Real MCP client-server communication
- ✅ **ADDED**: Async/await proper handling
- ✅ **ADDED**: Long-lived MCP sessions

### Labs:
- **Lab 0**: Complete MCP Setup
- **Lab 1**: MCP Architecture
- **Lab 2**: Connect to REAL MCP Servers
- **Lab 3**: Gmail via REAL MCP Server
- **Lab 4**: Calendar via REAL MCP Server

### 📧 Sender Email
**cygenaidemo@gmail.com**

---

# LAB 0: Complete MCP Setup for Google Colab

## Part A: Google Cloud Project Setup (Do this FIRST)

### Step 1: Create Google Cloud Project

Open this in a new tab: [Google Cloud Console](https://console.cloud.google.com/)

1. Click **"Select a project"** → **"New Project"**
2. Project Name: `mcp-colab-labs`
3. Click **"Create"**

### Step 2: Enable APIs

1. Go to **"APIs & Services"** → **"Library"**
2. Search and enable:
   - ✅ **Gmail API**
   - ✅ **Google Calendar API**

### Step 3: Create OAuth 2.0 Credentials

1. Go to **"APIs & Services"** → **"Credentials"**
2. Click **"Create Credentials"** → **"OAuth client ID"**
3. Configure OAuth consent screen (External)
4. Add scopes:
   - `https://www.googleapis.com/auth/gmail.send`
   - `https://www.googleapis.com/auth/gmail.readonly`
   - `https://www.googleapis.com/auth/calendar`
   - `https://www.googleapis.com/auth/calendar.events`
5. Add test user: `cygenaidemo@gmail.com`
6. Create OAuth client ID (Desktop app)
7. **Download JSON** file

## Part B: Install Node.js in Google Colab

In [ ]:
%%bash
# Install Node.js 20.x in Colab
echo "📦 Installing Node.js..."

curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
sudo apt-get install -y nodejs

echo ""
echo "✅ Node.js installed:"
node --version
npm --version

## Part C: Install Python Packages

In [ ]:
# Install Python packages
!pip install -q --pre -U langchain-openai langgraph langchain
!pip install -q python-dotenv
!pip install -q langchain-mcp-adapters mcp
!pip install -q google-auth google-auth-oauthlib google-auth-httplib2
!pip install -q google-api-python-client
!pip install -q nest_asyncio  # For async support in Colab

print("✅ Python packages installed (including nest_asyncio for async support)")

## Part D: Install MCP Servers

In [ ]:
%%bash
echo "📦 Installing REAL MCP servers from mcpservers.org..."
echo ""

echo "Installing mcp-google (Gmail + Calendar)..."
npm install -g mcp-google

echo ""
echo "Installing filesystem MCP server..."
npm install -g @modelcontextprotocol/server-filesystem

echo ""
echo "✅ MCP servers installed:"
npm list -g --depth=0 | grep -E "(mcp-google|modelcontextprotocol)"

## Part E: Upload OAuth Credentials

In [ ]:
from google.colab import files
import json
import os

print("📤 Upload your OAuth credentials JSON file")
print("   (The file you downloaded: client_secret_xxxxx.json)")
print()

uploaded = files.upload()

credentials_filename = list(uploaded.keys())[0]

!mkdir -p /content/mcp_config
!cp "{credentials_filename}" /content/mcp_config/credentials.json

with open('/content/mcp_config/credentials.json', 'r') as f:
    creds = json.load(f)
    cred_type = 'installed' if 'installed' in creds else 'web'
    client_id = creds[cred_type]['client_id']
    print(f"\n✅ Credentials uploaded successfully!")
    print(f"   Type: {cred_type}")
    print(f"   Client ID: {client_id[:30]}...")

print(f"\n📁 Credentials saved to: /content/mcp_config/credentials.json")

## Part F: Set Up Environment Variables

In [ ]:
import os
from getpass import getpass

try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ OpenAI API key loaded from Colab secrets")
except:
    print("⚠️ Colab secret not found. Enter manually:")
    OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

SENDER_EMAIL = "cygenaidemo@gmail.com"
CREDENTIALS_PATH = "/content/mcp_config/credentials.json"

print(f"\n📧 Sender Email: {SENDER_EMAIL}")
print(f"📁 Credentials: {CREDENTIALS_PATH}")

## Part G: Authenticate Google Services

In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
import json

print("🔐 Google Services Authentication (Gmail + Calendar)")
print("="*60)

GOOGLE_SCOPES = [
    'https://www.googleapis.com/auth/gmail.send',
    'https://www.googleapis.com/auth/gmail.readonly',
    'https://www.googleapis.com/auth/gmail.modify',
    'https://www.googleapis.com/auth/calendar',
    'https://www.googleapis.com/auth/calendar.events'
]

google_flow = InstalledAppFlow.from_client_secrets_file(
    CREDENTIALS_PATH,
    scopes=GOOGLE_SCOPES,
    redirect_uri='urn:ietf:wg:oauth:2.0:oob'
)

auth_url, _ = google_flow.authorization_url(prompt='consent')

print("\n🌐 Please visit this URL to authorize:")
print(auth_url)
print("\n📋 After authorizing, copy the authorization code and paste it below:")

code = input("Enter authorization code: ").strip()

google_flow.fetch_token(code=code)
google_creds = google_flow.credentials

token_path = '/content/mcp_config/google-token.json'
with open(token_path, 'w') as token:
    token.write(google_creds.to_json())

print(f"\n✅ Google services authenticated!")
print(f"📁 Token saved: {token_path}")

## Part H: Verify Setup

In [ ]:
import os

print("🔍 Setup Verification")
print("="*60)

checks = {
    "Node.js installed": os.system("which node > /dev/null 2>&1") == 0,
    "npm installed": os.system("which npm > /dev/null 2>&1") == 0,
    "mcp-google installed": os.system("npm list -g mcp-google > /dev/null 2>&1") == 0,
    "Credentials file": os.path.exists(CREDENTIALS_PATH),
    "Google token": os.path.exists('/content/mcp_config/google-token.json'),
    "OpenAI API key": os.getenv('OPENAI_API_KEY') is not None
}

for check, passed in checks.items():
    status = "✅" if passed else "❌"
    print(f"{status} {check}")

if all(checks.values()):
    print("\n🎉 All checks passed! Ready to proceed with labs.")
else:
    print("\n⚠️ Some checks failed. Please review the setup steps.")

## Part I: Create MCP Configuration

In [ ]:
import json

mcp_config = {
    "mcpServers": {
        "google-workspace": {
            "command": "npx",
            "args": ["-y", "mcp-google"],
            "env": {
                "GOOGLE_OAUTH_CREDENTIALS": "/content/mcp_config/credentials.json",
                "GOOGLE_CALENDAR_MCP_TOKEN_PATH": "/content/mcp_config/google-token.json"
            }
        },
        "filesystem": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-filesystem", "/content/data"],
            "env": {}
        }
    }
}

config_path = '/content/mcp_config.json'
with open(config_path, 'w') as f:
    json.dump(mcp_config, f, indent=2)

print("✅ MCP Configuration created")
print(f"📁 Config saved: {config_path}")
print("\n📋 Configuration:")
print(json.dumps(mcp_config, indent=2))

---

## ✅ Lab 0 Complete!

---

# Core Imports and Setup

In [ ]:
# Core imports
import os
import json
import asyncio
from pathlib import Path
from typing import List, Dict, Any
from datetime import datetime, timedelta
import nest_asyncio

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool

# LangGraph imports
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain.agents import create_agent

# MCP imports
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Enable nested event loops for Colab
nest_asyncio.apply()

print("✅ Imports complete (with async support)")
print(f"📧 Sender: {SENDER_EMAIL}")

## Sample Data

In [ ]:
!mkdir -p /content/data/resumes
!mkdir -p /content/data/contracts

HR_DATABASE = {
    "candidates": {
        "CAN001": {
            "name": "Priya Sharma",
            "email": "your-test-email@example.com",
            "position": "Senior Backend Engineer",
            "status": "screening"
        }
    }
}

SAMPLE_RESUME = """PRIYA SHARMA
Senior Software Engineer | 6+ years Python/AWS experience
Skills: FastAPI, microservices, Docker, Kubernetes"""

with open('/content/data/resumes/priya.txt', 'w') as f:
    f.write(SAMPLE_RESUME)

print("✅ Sample data loaded")

---

# LAB 1: MCP Architecture

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║         REAL MCP ARCHITECTURE IN GOOGLE COLAB               ║
╚══════════════════════════════════════════════════════════════╝

COLAB NOTEBOOK (Python)
      │
      │ MCP Client (stdio transport)
      │ ✅ REAL async communication
      ▼
┌─────────────────────┐
│ ClientSession       │  ← MCP Protocol
│ (Python)            │
└─────────┬───────────┘
          │
          │ stdio (stdin/stdout)
          │
    ┌─────┴─────┬──────────┐
    ▼           ▼          ▼
┌────────┐ ┌────────┐ ┌────────┐
│ Gmail  │ │Calendar│ │FileSys │
│  MCP   │ │  MCP   │ │  MCP   │
│ Server │ │ Server │ │ Server │
│(Node.js│ │(Node.js│ │(Node.js│
│process)│ │process)│ │process)│
└───┬────┘ └───┬────┘ └───┬────┘
    │          │          │
    ▼          ▼          ▼
[Gmail API][GCal API][/content/data]

✅ Using mcp-google from mcpservers.org
✅ Using @modelcontextprotocol/server-filesystem
✅ REAL MCP protocol (NOT simulation!)
✅ Long-lived MCP sessions
✅ Async/await handling with nest_asyncio
""")

with open('/content/mcp_config.json') as f:
    config = json.load(f)
    print("📋 Your MCP Configuration:")
    print(json.dumps(config, indent=2))

print("\n✅ Lab 1 Complete!")

---

# LAB 2: Connect to REAL MCP Servers

In [ ]:
# Global storage for MCP sessions
mcp_sessions = {}
mcp_contexts = {}

async def connect_to_mcp_server(server_name: str, server_config: dict):
    """
    Connect to a REAL MCP server and keep the session alive.
    """
    command = server_config['command']
    args = server_config['args']
    env = server_config.get('env', {})
    
    # Merge environment variables
    full_env = os.environ.copy()
    full_env.update(env)
    
    server_params = StdioServerParameters(
        command=command,
        args=args,
        env=full_env
    )
    
    print(f"🔌 Starting {server_name} MCP server process...")
    
    # Start the MCP server as a subprocess
    context = stdio_client(server_params)
    read, write = await context.__aenter__()
    
    # Store the context manager for cleanup later
    mcp_contexts[server_name] = context
    
    # Create session
    session = ClientSession(read, write)
    await session.initialize()
    
    # Store session globally
    mcp_sessions[server_name] = session
    
    # List available tools
    response = await session.list_tools()
    
    print(f"\n📦 {server_name} MCP Server Tools:")
    for tool_info in response.tools:
        print(f"  • {tool_info.name}: {tool_info.description[:80]}...")
    
    return session, response.tools

async def disconnect_all_mcp_servers():
    """Clean up all MCP server connections."""
    print("\n🔌 Disconnecting MCP servers...")
    for server_name, context in mcp_contexts.items():
        try:
            await context.__aexit__(None, None, None)
            print(f"  ✅ Disconnected {server_name}")
        except Exception as e:
            print(f"  ⚠️ Error disconnecting {server_name}: {e}")
    mcp_sessions.clear()
    mcp_contexts.clear()

print("✅ MCP connection functions ready")

In [ ]:
# Connect to MCP servers
async def initialize_mcp_servers():
    """Initialize all MCP servers from config."""
    with open('/content/mcp_config.json') as f:
        config = json.load(f)
    
    print("🚀 Initializing REAL MCP servers...\n")
    
    all_tools = {}
    for server_name, server_config in config['mcpServers'].items():
        try:
            session, tools = await connect_to_mcp_server(server_name, server_config)
            all_tools[server_name] = tools
            print(f"✅ {server_name}: Connected with {len(tools)} tools\n")
        except Exception as e:
            print(f"❌ {server_name}: Failed - {str(e)}\n")
    
    return all_tools

# Run initialization
print("⏳ Connecting to MCP servers (this may take 10-20 seconds)...\n")
available_tools = await initialize_mcp_servers()

print("\n🎉 All MCP servers are now running!")
print("\n✅ Lab 2 Complete!")

---

# LAB 3: Gmail via REAL MCP Server

In [ ]:
# Create wrapper function for REAL MCP Gmail tool
async def call_mcp_tool_async(server_name: str, tool_name: str, arguments: dict):
    """
    Call a tool on a REAL MCP server.
    """
    session = mcp_sessions.get(server_name)
    if not session:
        raise ValueError(f"MCP server {server_name} not connected")
    
    result = await session.call_tool(tool_name, arguments)
    return result

def call_mcp_tool_sync(server_name: str, tool_name: str, arguments: dict):
    """
    Synchronous wrapper for calling MCP tools (for use in LangChain).
    """
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(call_mcp_tool_async(server_name, tool_name, arguments))

# Create LangChain tool wrappers for Gmail MCP server
@tool
def send_gmail_via_real_mcp(to: str, subject: str, body: str) -> str:
    """
    Send email via REAL mcp-google server.
    
    Args:
        to: Recipient email address
        subject: Email subject line
        body: Email body text
    """
    try:
        result = call_mcp_tool_sync(
            server_name="google-workspace",
            tool_name="send_gmail",
            arguments={
                "to": to,
                "subject": subject,
                "body": body
            }
        )
        
        print(f"""
📧 EMAIL SENT VIA REAL MCP SERVER!
───────────────────────────────────────
From: {SENDER_EMAIL}
To: {to}
Subject: {subject}
MCP Server: mcp-google (REAL)
Result: {result.content[:100] if hasattr(result, 'content') else str(result)[:100]}...
───────────────────────────────────────
""")
        
        return f"✅ Email sent via REAL MCP server! Result: {result}"
    except Exception as e:
        return f"❌ Failed: {str(e)}"

@tool
def search_gmail_via_real_mcp(query: str, max_results: int = 10) -> str:
    """
    Search Gmail via REAL mcp-google server.
    
    Args:
        query: Search query
        max_results: Maximum number of results
    """
    try:
        result = call_mcp_tool_sync(
            server_name="google-workspace",
            tool_name="search_gmail",
            arguments={
                "query": query,
                "max_results": max_results
            }
        )
        return str(result)
    except Exception as e:
        return f"❌ Search failed: {str(e)}"

# Create agent with REAL MCP tools
gmail_tools = [send_gmail_via_real_mcp, search_gmail_via_real_mcp]
gmail_agent = create_agent(
    model="openai:gpt-4o",
    tools=gmail_tools,
    system_prompt=f"Gmail agent using REAL MCP server. Sender: {SENDER_EMAIL}"
)

print("✅ Gmail agent ready (using REAL MCP server)")
print(f"📧 Sender: {SENDER_EMAIL}")
print(f"🔌 Using: mcp-google server (REAL, not simulated)")

In [ ]:
# Test Gmail via REAL MCP
print("🧪 LAB 3: Gmail via REAL MCP Server\n" + "="*80)
print("⚠️ This will send a REAL email via the mcp-google server!\n")

TEST_EMAIL = "your-email@example.com"  # ← CHANGE THIS!

send_test_email = input(f"Send test email to {TEST_EMAIL}? (yes/no): ").lower() == 'yes'

if send_test_email:
    result = gmail_agent.invoke({
        "messages": [HumanMessage(content=f"""
Send interview invitation email to {TEST_EMAIL}.

Details:
- Candidate: Priya Sharma
- Position: Senior Backend Engineer
- Date: October 15, 2025 at 10:00 AM IST
- Interviewers: Rahul Verma, Vikram Singh
- Meeting: https://meet.google.com/test-interview

Make it professional and welcoming.
""")]
    })
    print("\n✅ Check your inbox!")
else:
    print("\n⏭️ Skipped. Change TEST_EMAIL and run again.")

print("\n✅ Lab 3 Complete!")

---

# LAB 4: Calendar via REAL MCP Server

In [ ]:
# Create LangChain tool wrappers for Calendar MCP server
@tool
def create_calendar_event_via_real_mcp(
    summary: str,
    start_time: str,
    end_time: str,
    attendees: str,
    description: str = "",
    location: str = ""
) -> str:
    """
    Create Google Calendar event via REAL mcp-google server.
    
    Args:
        summary: Event title
        start_time: ISO format (2025-10-15T10:00:00+05:30)
        end_time: ISO format
        attendees: Comma-separated emails
        description: Event description
        location: Location or link
    """
    try:
        result = call_mcp_tool_sync(
            server_name="google-workspace",
            tool_name="create_calendar_event",
            arguments={
                "summary": summary,
                "start": {"dateTime": start_time, "timeZone": "Asia/Kolkata"},
                "end": {"dateTime": end_time, "timeZone": "Asia/Kolkata"},
                "attendees": [email.strip() for email in attendees.split(',')],
                "description": description,
                "location": location
            }
        )
        
        print(f"""
📅 CALENDAR EVENT CREATED VIA REAL MCP SERVER!
───────────────────────────────────────
Event: {summary}
Start: {start_time}
MCP Server: mcp-google (REAL)
Result: {str(result)[:100]}...
───────────────────────────────────────
""")
        
        return f"✅ Event created via REAL MCP server! Result: {result}"
    except Exception as e:
        return f"❌ Failed: {str(e)}"

@tool
def list_calendar_events_via_real_mcp(max_results: int = 10) -> str:
    """
    List upcoming calendar events via REAL mcp-google server.
    
    Args:
        max_results: Maximum number of events to return
    """
    try:
        now = datetime.utcnow().isoformat() + 'Z'
        result = call_mcp_tool_sync(
            server_name="google-workspace",
            tool_name="list_calendar_events",
            arguments={
                "time_min": now,
                "max_results": max_results
            }
        )
        return str(result)
    except Exception as e:
        return f"❌ Failed: {str(e)}"

# Create agent with REAL MCP tools
calendar_tools = [create_calendar_event_via_real_mcp, list_calendar_events_via_real_mcp]
calendar_agent = create_agent(
    model="openai:gpt-4o",
    tools=calendar_tools,
    system_prompt="Calendar agent using REAL MCP server."
)

print("✅ Calendar agent ready (using REAL MCP server)")
print(f"🔌 Using: mcp-google server (REAL, not simulated)")

In [ ]:
# Test Calendar via REAL MCP
print("🧪 LAB 4: Calendar via REAL MCP Server\n" + "="*80)
print("⚠️ This will create a REAL calendar event via the mcp-google server!\n")

create_test_event = input("Create test calendar event? (yes/no): ").lower() == 'yes'

if create_test_event:
    result = calendar_agent.invoke({
        "messages": [HumanMessage(content=f"""
Create technical interview event:

- Title: Technical Interview - Priya Sharma
- Date: October 23, 2025
- Time: 10:00 AM - 11:30 AM IST
- Attendees: neependra.khare@gmail.com
- Location: https://meet.google.com/test-interview
- Description: Technical round (Python, AWS, System Design)
""")]
    })
    print("\n✅ Check your Google Calendar!")
else:
    print("\n⏭️ Skipped. Run again to create event.")

print("\n✅ Lab 4 Complete!")

---

# Cleanup: Disconnect MCP Servers

In [ ]:
# Always run this at the end to clean up MCP server processes
await disconnect_all_mcp_servers()
print("\n✅ All MCP servers disconnected")

---

# 🎉 Complete - REAL MCP Integration!

## What You Achieved:

✅ **REAL MCP Integration** - Not simulation!

✅ **Connected to actual MCP servers** via stdio transport

✅ **Long-lived MCP sessions** throughout agent execution

✅ **Async/await handling** with nest_asyncio

✅ **Real tool calls** through MCP protocol

## Key Differences from Simulation:

| Aspect | Simulation (Before) | Real MCP (Now) |
|--------|--------------------|-----------------|
| API Calls | Direct Google API | Through MCP Server |
| Process | Single Python process | Python + Node.js processes |
| Communication | N/A | stdio (stdin/stdout) |
| Sessions | N/A | Long-lived MCP sessions |
| Async | No | Yes (nest_asyncio) |

## 🔧 Architecture:

```
Python Agent → MCP Client → MCP Server (Node.js) → Google API
             (async)       (stdio)              (HTTP)
```

## 📚 Key Learnings:

1. **MCP Protocol** - How clients and servers communicate
2. **Async in Colab** - Using nest_asyncio for event loops
3. **Long-lived Sessions** - Keeping MCP connections open
4. **Tool Discovery** - Dynamic tool loading from servers

🚀 You now understand real MCP integration!